# Register embedding vectors for SEU morphologies in Nexus

## Setup

In [ ]:
import requests

In [ ]:
import getpass

In [ ]:
import uuid

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
from kgforge.specializations.mappings import DictionaryMapping

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
ENDPOINT = "http://embedder-bbp-dke-dev.ocp.bbp.epfl.ch"

In [ ]:
MODEL_NAME = "SEU_morph_attri2vec_euclidean"

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN, 
                            bucket="bbp-external/seu")

## Fetch all SEU neuron morphologies from Nexus to get id's

In [ ]:
query = """
    SELECT ?id ?rev ?name
    WHERE {
        ?id a NeuronMorphology ;
            name ?name ;
            <https://bluebrain.github.io/nexus/vocabulary/deprecated> false ;
            <https://bluebrain.github.io/nexus/vocabulary/rev> ?rev .
    }
"""

In [ ]:
resources = forge.sparql(query, limit=15000)

In [ ]:
len(resources)

In [ ]:
resource_dict = {r.id: r for r in resources}

## Get resource embeddings

In [ ]:
%%time
r = requests.get(
    f'{ENDPOINT}/models/{MODEL_NAME}/embedding',
    params={
        "resource_ids": list(resource_dict.keys())[:100]
    })
print(r)
embeddings = r.json()

In [ ]:
embeddings_list = list()

In [ ]:
for at_id, embedding in embeddings["vectors"].items():
    embeddings_list.append({
        "morphology_id": at_id,
        "morphology_rev": resource_dict[at_id].rev,
        "model_name": MODEL_NAME,
        "model_version": "v0",
        "embedding_name": f"Embedding of morphology {resource_dict[at_id].name} at revision {resource_dict[at_id].rev}",
        "embedding": embedding,
        "uuid": at_id.split("/")[-1]
        
    })

## Map embeddings to Nexus resources

In [ ]:
mapping = DictionaryMapping.load("./mappings/seu-embedding.hjson")

In [ ]:
embedding_resources = forge.map(embeddings_list, mapping)

In [ ]:
for r in embedding_resources:
    r.id = forge.format("identifier", "embeddings", str(uuid.uuid4()))

In [ ]:
# print(embedding_resources[0])

In [ ]:
forge_staging = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN, 
                            endpoint="https://staging.nexus.ocp.bbp.epfl.ch/v1",        
                            bucket="dke/seu-embeddings")

In [ ]:
for r in embedding_resources:
    forge_staging.register(r)